In [1]:
import sys   
import glob   
import serial
from serial import Serial

%matplotlib inline

def serial_ports():   
    """ Lists serial port names   
       
        :raises EnvironmentError:   
            On unsupported or unknown platforms   
        :returns:   
            A list of the serial ports available on the system   
    """   
    if sys.platform.startswith('win'):   
        ports = ['COM%s' % (i + 1) for i in range(256)]   
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):   
        # this excludes your current terminal "/dev/tty"   
        ports = glob.glob('/dev/tty[A-Za-z]*')   
    elif sys.platform.startswith('darwin'):   
        ports = glob.glob('/dev/tty.*')   
    else:   
        raise EnvironmentError('Unsupported platform')   
       
    result = []   
    for port in ports:   
        try:   
            s = serial.Serial(port)   
            s.close()   
            result.append(port)   
        except (OSError, serial.SerialException):   
            pass   
    return result   
   
if __name__ == '__main__':
    print(serial_ports())

['/dev/ttyUSB0']


In [2]:
ser = Serial('/dev/ttyUSB0',115200)

def serial_gps():
    for i in range(10):
        if ser.readable():
            res = ser.readline()
            msg=res.decode()[:len(res)-1]
            latitude = msg[10:18]
            longitude = msg[20:29]
            print(i, "Latitude :",latitude, "longitude:",longitude )
#             print(res.decode()[:len(res)-1]) # all information 

## file write test ##
f=open("test.txt",'w')
for i in range(10):
    if ser.readable():
            res = ser.readline()
            msg=res.decode()[:len(res)-1]
            latitude = msg[10:18]
            longitude = msg[20:29]
            print(i, "Latitude :",latitude, "longitude:",longitude )
    f.write(latitude+"  "+longitude+"\n")

f.close()
# serial_gps()

0 Latitude : nyGPS li longitude: ary v. 13
 longitude:  art
2 Latitude :  longitude: 
3 Latitude : Latitude longitude: Longitude
4 Latitude : (deg)    longitude: (deg)    
5 Latitude : -------- longitude: ---------
6 Latitude : ******** longitude: *********
7 Latitude : ******** longitude: *********
8 Latitude : ******** longitude: *********
9 Latitude : ******** longitude: *********


In [5]:
f=open("test.txt",'r')
data=f.read()
print(data)
f.close()

nyGPS li  ary v. 13
art
  
  
Latitude  Longitude
(deg)     (deg)    
--------  ---------
********  *********
********  *********
********  *********
********  *********



In [6]:
# GPS data save (Neo 6m GPS Module)
import time
saving_time=1*10
init=time.time()
f=open("gps-temp.txt",'w')
while True:
    if ser.readable():
        res = ser.readline()
        msg=res.decode()[:len(res)-1]
        latitude = msg[10:18]
        longitude = msg[20:29]
    f.write(latitude+"  "+longitude+"\n")
    time_=time.time()-init
    if time_> saving_time: break
f.close()

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import drawnow
import cv2

%matplotlib tk
points=(37.2156, 127.0673, 37.1630, 127.1311)
def scale_to_img(lat_lon, h_w):

    old = (points[2], points[0])
    new = (0, h_w[1])
    y = ((lat_lon[0] - old[0]) * (new[1] - new[0]) / (old[1] - old[0])) + new[0]
    old = (points[1], points[3])
    new = (0, h_w[0])
    x = ((lat_lon[1] - old[0]) * (new[1] - new[0]) / (old[1] - old[0])) + new[0]
    # y must be reversed because the orientation of the image in the matplotlib.
    # image - (0, 0) in upper left corner; coordinate system - (0, 0) in lower left corner
    return int(x), h_w[1] - int(y)

def create_img(color, width=2):
    data = pd.read_csv('gps.csv', names=['LATITUDE', 'LONGITUDE'], sep=',')

    result_image = Image.open('map_1.png', 'r')
    img_points = []
    gps_data = tuple(zip(data['LATITUDE'].values, data['LONGITUDE'].values))
    print(len(gps_data))
    for d in gps_data[:50]:
        x1, y1 = scale_to_img(d, (result_image.size[0], result_image.size[1]))
        img_points.append((x1, y1))
        draw = ImageDraw.Draw(result_image)
        draw.line(img_points, fill=color, width=width)
    #drawing line !!!
    draw = ImageDraw.Draw(result_image)
    draw.line(img_points, fill=color, width=width)
    
    return result_image

def get_ticks():
    x_ticks = []
    y_ticks = []
    x_ticks = map(lambda x: round(x, 4),np.linspace(points[1], points[3], num=7))
    y_ticks = map(lambda x: round(x, 4), np.linspace(points[2], points[0], num=8))
    y_ticks = sorted(y_ticks, reverse=True)
    
    return x_ticks,y_ticks

904


In [49]:
data = pd.read_csv('gps.csv', names=['LATITUDE', 'LONGITUDE'], sep=',')
result_image = Image.open('map_1.png', 'r')
img_points = []
gps_data = tuple(zip(data['LATITUDE'].values, data['LONGITUDE'].values))

for d in gps_data:
    x1, y1 = scale_to_img(d, (result_image.size[0], result_image.size[1]))
    img_points.append((x1, y1))
    draw = ImageDraw.Draw(result_image)
    draw.line(img_points, (0,0,255), width=3)
    cv2_img=np.array(result_image)
    # img = cv2.imread('map_1.png', cv2.IMREAD_COLOR)
    img1= cv2.cvtColor(cv2_img, cv2.COLOR_RGB2BGR)
    cv2.namedWindow('Navigation', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Navigation', 800, 600)   
    cv2.imshow("Navigation",img1)
    if cv2.waitKey(1)==27:break
    time.sleep(0.1)

cv2.destroyAllWindows()

In [4]:
result_image1 = Image.open('map_1.png', 'r')
img_points1=[]
c=0
b=0
while True:
    if ser.readable():
        res = ser.readline()
        msg=res.decode()[:len(res)-1]
#         latitude = msg[10:18]
#         longitude = msg[20:29]
        latitude = 37.2
        longitude = 127.2
        print(c, "Latitude :",latitude, "longitude:",longitude )
        
    try:
        lat=float(latitude)
        logt=float(longitude)
        d=(lat,logt)
        
        x1, y1 = scale_to_img(d, (result_image.size[0], result_image.size[1]))
#         print(x1,y1)
        img_points1.append((x1, y1))
        draw = ImageDraw.Draw(result_image1)
        draw.line(img_points1, (0,0,255), width=3)
        cv2_img=np.array(result_image1)
        # img = cv2.imread('map_1.png', cv2.IMREAD_COLOR)
 
    except:
        b+=1
        
    print("check!")
    cv2_img1=np.array(result_image1)
    img1= cv2.cvtColor(cv2_img1, cv2.COLOR_RGB2BGR)
    cv2.namedWindow('Navigation', cv2.WINDOW_AUTOSIZE)
    cv2.imshow("Navigation",img1)
    if cv2.waitKey(1)==27:break
    time.sleep(0.01)
    c+=1
    
cv2.destroyAllWindows()

0 Latitude : 37.2 longitude: 127.2
check!


NameError: name 'img1' is not defined